In [1]:
import sys
sys.path.append('generate_data')
from generator import *
import pandas as pd
pd_v = pd.__version__
python_v = !python -V
print("version", *python_v)
print("version Pandas", pd_v)

version Python 3.10.5
version Pandas 2.0.2


**Входные данные**

In [6]:
!python generate_data/generator.py

In [2]:
df_customers = read_csv(os.path.join(output_location, 'customers.csv'))
df_customers.head()

,customer_id,loyalty_score
0,C1,7
1,C2,4
2,C3,13
3,C4,11
4,C5,8


In [3]:
df_products = read_csv(os.path.join(output_location, 'products.csv'))
df_products.head()

,product_id,product_description,product_category
0,P01,detergent,house
1,P02,kitchen roll,house
2,P03,bin liners,house
3,P04,shower gel,house
4,P05,scented candles,house


In [4]:
df_transactions = read_json_folder(os.path.join(output_location, 'transactions',''))
df_transactions.head()

,customer_id,basket,date_of_purchase
0,C1,"[{'product_id': 'P63', 'price': 1674}]",2023-03-28 00:39:30.397598
1,C4,"[{'product_id': 'P34', 'price': 449}]",2023-03-28 15:22:30.397598
2,C6,"[{'product_id': 'P63', 'price': 401}]",2023-03-28 13:27:30.397598
3,C15,"[{'product_id': 'P01', 'price': 1224}, {'produ...",2023-03-28 06:31:30.397598
4,C28,"[{'product_id': 'P59', 'price': 1264}, {'produ...",2023-03-28 00:28:30.397598


**Решение**

In [11]:
# Выбор определенного периода, за который необходимо собрать данные
time_frame = ['2023-03-28', '2023-06-28'] # Например


def to_date(one_date):
    return datetime.fromisoformat(one_date).date()


df_transactions = df_transactions.loc[(df_transactions.date_of_purchase.apply(to_date) >= to_date(time_frame[0])) \
                                      & (df_transactions.date_of_purchase.apply(to_date) <= to_date(time_frame[1]))]
df_transactions.head()

,customer_id,basket,date_of_purchase
0,C1,"[{'product_id': 'P63', 'price': 1674}]",2023-03-28 00:39:30.397598
1,C4,"[{'product_id': 'P34', 'price': 449}]",2023-03-28 15:22:30.397598
2,C6,"[{'product_id': 'P63', 'price': 401}]",2023-03-28 13:27:30.397598
3,C15,"[{'product_id': 'P01', 'price': 1224}, {'produ...",2023-03-28 06:31:30.397598
4,C28,"[{'product_id': 'P59', 'price': 1264}, {'produ...",2023-03-28 00:28:30.397598


In [6]:
# Распаковка колонки 'basket'
df = df_transactions.explode('basket').reset_index(drop=True)
split_df = pd.DataFrame(df['basket'].to_list(), columns = ['product_id'])
df = pd.concat([df, split_df], axis=1)
df = df.drop('basket', axis=1)

In [7]:
# Подсчет количества 
df = df.groupby(['customer_id','product_id']).count().reset_index()


In [8]:
# Добавление параметров 'loyalty_score' и 'product_category'
df = pd.merge(df, df_customers[['customer_id', 'loyalty_score']], on='customer_id')
df = pd.merge(df, df_products[['product_id', 'product_category']], on='product_id')
df.rename(columns={'date_of_purchase': 'purchase_count'}, inplace=True)
df = df[['customer_id', 'loyalty_score', 'product_id', 'product_category', 'purchase_count']]
df.product_category.unique() 
# Обрезать категорию по первой букве не корректо, есть категории с одинаковой первой буквой 
# df.product_category.apply(lambda x: x[0].upper())

array(['house', 'clothes', 'fruit_veg', 'food', 'bws', 'sweets'],
      dtype=object)

In [9]:
# Результат
df

,customer_id,loyalty_score,product_id,product_category,purchase_count
0,C1,7,P01,house,1
1,C105,8,P01,house,3
2,C121,9,P01,house,4
3,C125,1,P01,house,7
4,C128,8,P01,house,2
...,...,...,...,...,...
2238,C77,14,P20,clothes,2
2239,C80,2,P20,clothes,2
2240,C91,2,P20,clothes,1
2241,C97,4,P20,clothes,1
